# **MÓDULO 35 - Cross Validation**

Nesta tarefa, você trabalhará com uma base de dados que contém informações sobre variáveis ambientais coletadas para a detecção de incêndios. O objetivo é utilizar técnicas de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação na previsão da ocorrência de um incêndio com base nas variáveis fornecidas.


Descrição da Base de Dados
A base de dados contém as seguintes variáveis:

Unnamed:0: Índice (não é uma variável útil para o modelo)

UTC: Tempo em Segundos UTC

Temperature[C]: Temperatura do Ar (em graus Celsius)

Humidity[%]: Umidade do Ar (em porcentagem)

TVOC[ppb]: Total de Compostos Orgânicos Voláteis (medido em partes por bilhão)

eCO2[ppm]: Concentração equivalente de CO2 (medido em partes por milhão)

Raw H2: Hidrogênio molecular bruto, não compensado

Raw Ethanol: Etanol gasoso bruto

Pressure[hPA]: Pressão do Ar (em hectopascais)

PM1.0: Material particulado de tamanho < 1,0 µm

PM2.5: Material particulado de tamanho >1,0 µm e < 2,5 µm

NC0.5: Concentração numérica de material particulado de tamanho < 0,5 µm

NC1.0: Concentração numérica de material particulado de tamanho 0,5 µm < 1,0 µm

NC2.5: Concentração numérica de material particulado de tamanho 1,0 µm < 2,5 µm

CNT: Contador de amostras


E a variável alvo:

Fire Alarm: Indicador binário de incêndio (1 se houver incêndio, 0 caso contrário)

O objetivo desta tarefa é aplicar a técnica de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação. A validação cruzada ajudará a garantir que o modelo seja avaliado de maneira robusta e generalize bem para dados não vistos.

In [6]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 1 - Carregue a base de dados, verifique os tipos de dados e também se há presença de dados faltantes ou nulos.

In [7]:
data = pd.read_csv('/content/Cientista de dados M35 - smoke_detection_iot.csv')

print('_____________________________________________________________________________________________________\n')

print('Valores nulos presentes no dataframe\n')
print(data.isnull().sum())

print('_____________________________________________________________________________________________________\n')

print('Tipos de dados presentes no dataframe\n')
print(data.info())

print('_____________________________________________________________________________________________________\n')

print('Primeiras linhas do dataframe\n')
print(data.head())

print('_____________________________________________________________________________________________________\n')

print('Últimas linhas do dataframe\n')
print(data.tail())

print('_____________________________________________________________________________________________________\n')

print('Distribuição da variável alvo (Fire Alarm)\n')
print(data['Fire Alarm'].value_counts())

_____________________________________________________________________________________________________

Valores nulos presentes no dataframe

Unnamed: 0        0
UTC               0
Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5             0
NC1.0             0
NC2.5             0
CNT               0
Fire Alarm        0
dtype: int64
_____________________________________________________________________________________________________

Tipos de dados presentes no dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      62630 non-null  int64  
 1   UTC             62630 non-null  int64  
 2   Temperature[C]  62630 non-null  float64
 3   Humidity[%]     62630 non-null  float64
 4   

Para a coluna Fire Alarm, por conta do espaçamento talvez seja util renomear o nome da coluna utilizando:

df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

# 2 - Para essa base, onde você realizará as previsões de fire alarm, qual modelo de machine learning você aplicará? Justifique.




Regressão Logística

Justificativa: A variável alvo 'Fire Alarm' é binária (0 ou 1), indicando um problema de classificação binária.

A Regressão Logística é um modelo adequado e comum para este tipo de problema, pois prevê a probabilidade de uma ocorrência binária com base nas variáveis preditoras. Além disso, é um modelo interpretável e computacionalmente eficiente.


# 3 - Separe a base em Y e X e já rode a instância do modelo que você utilizará.

In [8]:
from sklearn.preprocessing import StandardScaler

data.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

# Separar as variáveis preditoras (X) e a variável alvo (Y)
X = data.drop(['Unnamed: 0', 'UTC', 'CNT', 'Fire_Alarm'], axis=1)
Y = data['Fire_Alarm']

# Padronizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Instância do modelo
model = LogisticRegression(max_iter=1000)

# 4 - Defina o número de Folds e rode o modelo com a validação cruzada.

In [9]:
# Defininido o número de folds
n_splits = 5

# Instanciar o KFold
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Definir métricas de avaliação
scoring = ['accuracy', 'roc_auc', 'recall', 'f1']

# Executar a validação cruzada com múltiplas métricas
scores = cross_val_score(model, X_scaled, Y, cv=kfold, scoring='accuracy') # Keep accuracy for comparison
roc_auc_scores = cross_val_score(model, X_scaled, Y, cv=kfold, scoring='roc_auc')
recall_scores = cross_val_score(model, X_scaled, Y, cv=kfold, scoring='recall')
f1_scores = cross_val_score(model, X_scaled, Y, cv=kfold, scoring='f1')

# Imprimir os resultados da validação cruzada para acurácia
print(f'Scores de Acurácia para cada fold: {scores}')
print(f'Acurácia média: {scores.mean():.4f}')
print(f'Desvio padrão dos scores de Acurácia: {scores.std():.4f}')

print('_____________________________________________________________________________________________________\n')

# Imprimir os resultados da validação cruzada para ROC AUC
print(f'Scores de ROC AUC para cada fold: {roc_auc_scores}')
print(f'ROC AUC média: {roc_auc_scores.mean():.4f}')
print(f'Desvio padrão dos scores de ROC AUC: {roc_auc_scores.std():.4f}')

print('_____________________________________________________________________________________________________\n')

# Imprimir os resultados da validação cruzada para Recall
print(f'Scores de Recall para cada fold: {recall_scores}')
print(f'Recall médio: {recall_scores.mean():.4f}')
print(f'Desvio padrão dos scores de Recall: {recall_scores.std():.4f}')

print('_____________________________________________________________________________________________________\n')

# Imprimir os resultados da validação cruzada para F1-Score
print(f'Scores de F1-Score para cada fold: {f1_scores}')
print(f'F1-Score médio: {f1_scores.mean():.4f}')
print(f'Desvio padrão dos scores de F1-Score: {f1_scores.std():.4f}')

Scores de Acurácia para cada fold: [0.89541753 0.89517803 0.89262334 0.89765288 0.89988823]
Acurácia média: 0.8962
Desvio padrão dos scores de Acurácia: 0.0025
_____________________________________________________________________________________________________

Scores de ROC AUC para cada fold: [0.96465336 0.96538168 0.96600176 0.96485976 0.96596493]
ROC AUC média: 0.9654
Desvio padrão dos scores de ROC AUC: 0.0006
_____________________________________________________________________________________________________

Scores de Recall para cada fold: [0.94995522 0.95181533 0.94628928 0.95448446 0.95670562]
Recall médio: 0.9518
Desvio padrão dos scores de Recall: 0.0036
_____________________________________________________________________________________________________

Scores de F1-Score para cada fold: [0.92833698 0.92825529 0.92661902 0.93014385 0.93201778]
F1-Score médio: 0.9291
Desvio padrão dos scores de F1-Score: 0.0018


# 5 - Avalie a pontuação de cada modelo e ao final a validação final da média.

In [10]:
# Como visto acima

print("\nAvaliação da pontuação de cada modelo e a validação final da média:")
for i in range(n_splits):
  print(f"Fold {i+1}:")
  print(f"  Acurácia: {scores[i]:.4f}")
  print(f"  ROC AUC: {roc_auc_scores[i]:.4f}")
  print(f"  Recall: {recall_scores[i]:.4f}")
  print(f"  F1-Score: {f1_scores[i]:.4f}")

print(f"\nAcurácia média da validação cruzada: {scores.mean():.4f}")
print(f"Desvio padrão dos scores da validação cruzada: {scores.std():.4f}")

print(f"\nROC AUC média da validação cruzada: {roc_auc_scores.mean():.4f}")
print(f"Desvio padrão dos scores de ROC AUC da validação cruzada: {roc_auc_scores.std():.4f}")

print(f"\nRecall médio da validação cruzada: {recall_scores.mean():.4f}")
print(f"Desvio padrão dos scores de Recall da validação cruzada: {recall_scores.std():.4f}")

print(f"\nF1-Score médio da validação cruzada: {f1_scores.mean():.4f}")
print(f"Desvio padrão dos scores de F1-Score da validação cruzada: {f1_scores.std():.4f}")


Avaliação da pontuação de cada modelo e a validação final da média:
Fold 1:
  Acurácia: 0.8954
  ROC AUC: 0.9647
  Recall: 0.9500
  F1-Score: 0.9283
Fold 2:
  Acurácia: 0.8952
  ROC AUC: 0.9654
  Recall: 0.9518
  F1-Score: 0.9283
Fold 3:
  Acurácia: 0.8926
  ROC AUC: 0.9660
  Recall: 0.9463
  F1-Score: 0.9266
Fold 4:
  Acurácia: 0.8977
  ROC AUC: 0.9649
  Recall: 0.9545
  F1-Score: 0.9301
Fold 5:
  Acurácia: 0.8999
  ROC AUC: 0.9660
  Recall: 0.9567
  F1-Score: 0.9320

Acurácia média da validação cruzada: 0.8962
Desvio padrão dos scores da validação cruzada: 0.0025

ROC AUC média da validação cruzada: 0.9654
Desvio padrão dos scores de ROC AUC da validação cruzada: 0.0006

Recall médio da validação cruzada: 0.9518
Desvio padrão dos scores de Recall da validação cruzada: 0.0036

F1-Score médio da validação cruzada: 0.9291
Desvio padrão dos scores de F1-Score da validação cruzada: 0.0018
